In [1]:
pip install jdcal

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [39]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import datetime
import os
from astropy.timeseries import LombScargle 
from jdcal import gcal2jd
# from CosinorPy import cosinor, cosinor1

In [41]:
def data_orgaziner(file_name):

    col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]
    
    for i in range(1, 33):
        col_names.append(f"Sp{i}")
    
    folder_path = 'Data'
    file_path = os.path.join(folder_path, file_name)
    
    df = pd.read_csv(file_path, names=col_names, sep='\s+', header=None)
    df = df.set_index('Index')
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
    deleted_data = df[df["MonStatus"] != 1]
    print(f"Removed rows of data where Monitor Status is not 1: {len(deleted_data)}\n")
    df = df[df["MonStatus"] == 1]
    
    month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    df['DateM'] = df['DateM'].str[:3].map(month_map)
    df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
    df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')
    
    df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year,
                                         month=df['Date'].dt.month,
                                         day=df['Date'].dt.day,
                                         hour=df['Time'].dt.hour,
                                         minute=df['Time'].dt.minute,
                                         second=df['Time'].dt.second))
    
    df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)
    
    day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}
    
    df.insert(0, 'Day', df['Time'].dt.day.map(day_map))
    
    spiders = []
    
    for i in range(1, 33):
        if df[f"Sp{i}"].sum() > 10:
            spiders.append(i)
        if not i in spiders:
            df = df.drop([f"Sp{i}"], axis=1)

    return df, spiders

def naming_pattern(file_name):
    group_name = file_name.split(' ', 2)[0]
    light_condition = file_name.split(' ', 2)[1]
    start_date = file_name.split(' ', 2)[2].split('-', 1)[0]
    
    path = group_name + "_" + light_condition + "_" + start_date
    
    two_lights = False
    if "-" in light_condition:
        #li_con_1, li_con_2 = light_condition.split('-', 1)
        two_lights = True

    return group_name, light_condition, start_date, path, two_lights

def zeros_counter(df):
    pass

In [43]:
# folder_path = 'Data'
# files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# zeros_dict = {}
# for file_name in files:
#     print(file_name, "\n")
#     df, spiders = data_orgaziner(file_name)

#     zeros = []
#     for i in spiders:
#         zerosTF = df[f"Sp{i}"] == 0
#         zeros.append(zerosTF.sum())
    
#     zeros_avg = sum(zeros) / len(zeros)
#     zeros_percent = zeros_avg / len(df)
#     zeros_dict[file_name.split(".", 2)[0]] = zeros_percent

# # zeros_dict["Average across all"] = 
# zeros_frame = pd.DataFrame.from_dict(zeros_dict, orient='index', columns=['Percent of zeros'])
# # averages = df.mean(axis=1)
# zeros_frame.loc['Average across all'] = zeros_frame['Percent of zeros'].mean()
# display(zeros_frame)


# zeros_frame.to_csv('zero_percents_all_data.txt', sep='\t', header=False, index=True)

In [45]:
file_name = 'spiders in red light.txt'

df, spiders = data_orgaziner(file_name)

display(df)

group_name, light_condition, start_date, path, two_lights = naming_pattern(file_name)



FileNotFoundError: [Errno 2] No such file or directory: 'Data/spiders in red light.txt'

# Light code

In [48]:
def light_code(df):
    LL_days = []
    LD_days = []
    
    for j in range(1, len(df['Day'].unique()) + 1):
        curr_df = df[df['Day']==j]
    
        if curr_df['Light'].sum() > 10:
            if curr_df['Light'].sum() > 1000:
                LL_days.append(j)
            else:
                LD_days.append(j)
    
    condition_days = {}
    
    condition_days['LD'] = LD_days
    
    condition_days['DD'] = [x for x in df['Day'].unique() if not x in LL_days and not x in LD_days]
    
    condition_days['LL'] = LL_days
    
    
    """
    if two_lights:
        name_wrong = li_con_1 == 
        
    if light_condition == 
    
    li_con_1, li_con_2
    """
    
    condition_keys = [key for key in condition_days if condition_days[key]]

    return condition_days, condition_keys
    


In [50]:
condition_days, condition_keys = light_code(df)
print(condition_days)

{'LD': [1, 11], 'DD': [], 'LL': [2, 3, 4, 5, 6, 7, 8, 9, 10]}


# Raster Plot Code

In [52]:
df_res = df.copy()
df_res.set_index('Time', inplace=True)

spider_columns = [f"Sp{sp}" for sp in spiders]

df_resampled = df_res[spider_columns].resample('6T').sum()

def convert_to_one(x):
    return 1 if x != 0 else 0

df_binary = df_resampled[spider_columns].map(convert_to_one)

day_resampled = df_res['Day'].resample('6T').bfill()
light_resampled = df_res['Light'].resample('6T').bfill()

df_binary.insert(0, "Day", day_resampled)
df_binary.insert(1, "Light", light_resampled)
display(df_binary)

##############

fig22 = plt.figure(figsize=(6, 4))

for j in range(1, len(df['Day'].unique()) + 1):
    curr_df = df_binary[df_binary['Day']==j]

    ax = fig22.add_subplot(len(df['Day'].unique()), 1, j)
    
    ax.set_ylabel(j, rotation="horizontal", va="center", ha="right", fontsize=8)
    
    for k in range(len(curr_df) - 1):
        start_time = curr_df.index[k]
        end_time = curr_df.index[k + 1]
        if curr_df['Light'].iloc[k] == 1:
            ax.axvspan(start_time.hour + start_time.minute / 60, 
                       end_time.hour + end_time.minute / 60, 
                       color='yellow', alpha=0.3)
    
    ax.bar(curr_df.index.hour + curr_df.index.minute / 60, 
           curr_df["Sp3"], width=0.05, align='center')
    
    ax.set_ylim(0, 1.1)
    ax.tick_params(left=False, bottom=False)
    ax.set_yticklabels([])
    ax.set_xlim(-0.5, 23.5)
    ax.set_xticklabels([])
    
    ax.set_xlabel("")


    if j == 1:
        ax.set_title(f"{group_name} 17")

    if j == len(df["Day"].unique()):
        ax.set_xticks(np.arange(0, 25, 2))
        ax.set_xticklabels(np.arange(0, 25, 2), rotation = 'horizontal', fontsize = 7)
        ax.set_xlabel('Time (hours)')

    """
    
    if j == condition_days[light_con][0]:
        ax.set_title(f"{group_name} {sp.split('p', 2)[1]}")
        #ax.text(0.5, 1.5, x, fontsize=12, horizontalalignment='center', transform=ax.transAxes)
            
    if j == len(df["Day"].unique()):
        ax.set_xticks(np.arange(0, 25, 2))
        ax.set_xticklabels(np.arange(0, 25, 2), rotation = 'horizontal', fontsize = 7)
        ax.set_xlabel('Time (hours)')

    """



plt.show()



NameError: name 'spiders' is not defined

In [29]:
def raster_plot_2(sp, fig, light_con):
    for j in condition_days[light_con]:
        curr_df = df[df['Day']==j]

        curr_df_avg = curr_df.groupby(curr_df['Time'].dt.hour)[sp].mean()
    
        ax = fig.add_subplot(len(df['Day'].unique()), 1, j)
        ax.set_ylabel(f"Day {j}", rotation="horizontal", va="center", ha="right", fontsize=8)

        
        for k in range(0, len(curr_df) -1):
            start_time = curr_df['Time'].iloc[k]
            end_time = curr_df['Time'].iloc[k + 1]
            if curr_df['Light'].iloc[k] == 1:
                ax.axvspan(start_time.hour + start_time.minute / 60, end_time.hour + end_time.minute / 60, color='yellow', alpha=0.3)

        curr_df_avg.plot.bar(y = [sp], ax=ax)
        
        ax.set_ylim(0, 1)
        ax.legend().remove()
        ax.tick_params(left=False, bottom=False)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        ax.set_xlim(-0.5, 23.5)
        ax.set_xlabel("")
        
        if j == condition_days[light_con][0]:
            ax.set_title(f"{group_name} {sp.split('p', 2)[1]}")
            #ax.text(0.5, 1.5, x, fontsize=12, horizontalalignment='center', transform=ax.transAxes)
            
        if j == len(df["Day"].unique()):
            ax.set_xticks(np.arange(0, 25, 2))
            ax.set_xticklabels(np.arange(0, 25, 2), rotation = 'horizontal', fontsize = 7)
            ax.set_xlabel('Time (hours)')
        

    """
    file_path = os.path.join(raster_path, f"{group_name}_{sp.split('p', 2)[1]}_{light_con}_in_{light_condition}_{start_date}_raster_plot.png")
    plt.savefig(file_path)
    """
    #plt.close()
    plt.show()

In [31]:
def raster_plot(sp, fig, light_con):
    for j in condition_days[light_con]:
        curr_df = df[df['Day']==j]

        #curr_df_avg = curr_df.groupby(curr_df['Time'].dt.hour)[sp].mean()
    
        ax = fig.add_subplot(len(df['Day'].unique()), 1, j)
        ax.set_ylabel(j, rotation="horizontal", va="center", ha="right", fontsize=8)

        
        for k in range(0, len(curr_df) -1):
            start_time = curr_df['Time'].iloc[k]
            end_time = curr_df['Time'].iloc[k + 1]
            if curr_df['Light'].iloc[k] == 1:
                ax.axvspan(start_time.hour * 60 + start_time.minute, end_time.hour * 60 + end_time.minute, color='yellow', alpha=0.3)

        curr_df.plot.bar(y = [sp], ax=ax)
        
        ax.set_ylim(0, 5)
        ax.legend().remove()
        ax.tick_params(left=False, bottom=False)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        ax.set_xlim(-0.5, 1440.5)
        ax.set_xlabel("")
        
        if j == condition_days[light_con][0]:
            ax.set_title(f"{group_name} {sp.split('p', 2)[1]}")
            #ax.text(0.5, 1.5, x, fontsize=12, horizontalalignment='center', transform=ax.transAxes)
            
        if j == len(df["Day"].unique()):
            ax.set_xticks(np.arange(0, 25, 2))
            ax.set_xticklabels(np.arange(0, 25, 2), rotation = 'horizontal', fontsize = 7)
            ax.set_xlabel('Time (hours)')
        
    file_path = os.path.join(raster_path, f"{group_name}_{sp.split('p', 2)[1]}_{light_con}_in_{light_condition}_{start_date}_raster_plot.png")
    plt.savefig(file_path)
    
    plt.close()
    #plt.show()

In [33]:
raster_path = path + "_raster_plots"

if not os.path.exists(raster_path):
    os.makedirs(raster_path)




x = "Sp17"

fig1 = plt.figure(figsize=(4, 6))
raster_plot(x, fig1, 'DD')

#fig2 = plt.figure(figsize=(4, 6))
#raster_plot_2(x, fig2, 'DD')

"""

for light in condition_keys:
    for i in spiders:
        fig = plt.figure(figsize=(4, 6))
        raster_plot(f"Sp{i}", fig, light)

"""


"""
KhA LD 0604-0607-2024

Little data spiders: 4, 10, 11, 12, 14

Not at the cork spiders (beginning): 4, 9, 11, 12, 13, 14

"""


NameError: name 'path' is not defined

In [54]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import datetime
import os

col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]

for i in range(1, 33):
    col_names.append(f"Sp{i}")

file = 'spiders in red light.txt'

df = pd.read_csv(file, names=col_names, sep='\s+', header=None)
df = df.set_index('Index')
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')


month_map = {'Jan': 1, 'Feb': 2, 'Mar' : 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
df['DateM'] = df['DateM'].str[:3].map(month_map)
df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')

df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year,
                                     month=df['Date'].dt.month,
                                     day=df['Date'].dt.day,
                                     hour=df['Time'].dt.hour,
                                     minute=df['Time'].dt.minute,
                                     second=df['Time'].dt.second))

df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)

day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}

df.insert(0, 'Day', df['Time'].dt.day.map(day_map))

#df.to_csv('steatodaA.csv')
display(df)

def datetime_to_jd(dt):
    year, month, day = dt.year, dt.month, dt.day
    hour, minute, second = dt.hour, dt.minute, dt.second
    _, jd = gcal2jd(year, month, day)
    jd += (hour + minute / 60 + second / 3600) / 24
    return jd

# value, display, save, dis+save

def period_LS(df, sp, light_con, result_type='value'):
    if result_type != 'value' and result_type != 'display' and result_type != 'save' and result_type != 'dis+save':
        print("The 'result_type' parameter only takes in 'value', 'display', 'save', 'dis+save'.")
        return None
    
    first_day = condition_days[light_con][0]
    last_day = condition_days[light_con][-1]
    
    curr_df = df[(df["Day"] >= first_day) & (df["Day"] <= last_day)]

    activity = np.array(curr_df[sp])
    time = curr_df['Time'].apply(datetime_to_jd)

    ls = LombScargle(time, activity)
    freq = np.linspace(0.5, 2, len(activity))
    power = ls.power(freq)
    
    max_power = np.max(power)
    max_freq = freq[np.argmax(power)]

    periods = 1 / freq * 24
    power_new = power.copy()

    fap = ls.false_alarm_probability(max_power)

    #print(fap)

    """
    
    while max_freq > 1.9 or max_freq < 0.75:
        max_power_indices = np.where(power_new == max_power)[0]
        power_new = np.delete(power_new, max_power_indices)
        #periods = np.delete(periods, max_power_indices)
        
        max_power = np.max(power_new)
        max_freq = freq[np.argmax(power_new)]

    """

    diff = np.diff(power)
    local_maxima_indices = np.where(np.logical_and(diff[:-1] > 0, diff[1:] < 0))[0]

    
    local_maxima_freqs = freq[local_maxima_indices]
    
    sorted_indices = np.argsort(power[local_maxima_indices])[::-1]
    
    sorted_local_maxima_freqs = local_maxima_freqs[sorted_indices]

    period1, period2, period3 = 24 / sorted_local_maxima_freqs[:3] 

    print("Periods under top 3 local maxima frequencies:")
    print("1:", period1)
    print("2:", period2)
    print("3:", period3)
    
    period = 1 / max_freq
    period *= 24
    #print(f"The Lomb-Scargle approximation of the period is {period} hours.")


    if result_type != 'value':
        plt.figure(figsize=(10, 6))
        plt.plot(periods, power)
        plt.scatter(period1, max_power, s=75, c="red")
        #plt.scatter(period2, max_power, s=75, c="red")
        plt.xlabel('Period (hours)')
        plt.ylabel('Power')
        plt.xlim(12, 36)
        plt.xticks(np.arange(12, 37, 2))
        plt.title(f'Lomb-Scargle Periodogram for {group_name} {sp.split("p", 2)[1]}')
        
        if result_type == 'display':
            plt.show()

        elif result_type == 'save':
            file_path = os.path.join(LS_path, f"LS_{group_name}_{sp.split('p', 2)[1]}_{light_condition}_{start_date}.png")
            plt.savefig(file_path)
            plt.close()

        else: # result_type == 'dis+save'
            file_path = os.path.join(LS_path, f"LS_{group_name}_{sp.split('p', 2)[1]}_{light_condition}_{start_date}.png")
            plt.savefig(file_path)
            plt.show()

    return period, fap



Day                Time  Light  Sp1  Sp2  Sp3  Sp4  Sp5  Sp6  Sp7  ...  \
Index                                                                     ...   
19       1 2024-06-18 15:39:00      1    0    0    0    0    0    0    2  ...   
20       1 2024-06-18 15:40:00      1    0    0    0    0    0    0    0  ...   
21       1 2024-06-18 15:41:00      1    0    0    0    0    0    0    0  ...   
22       1 2024-06-18 15:42:00      1    0    0    0    0    0    0    0  ...   
23       1 2024-06-18 15:43:00      1    0    0    0    0    0    0    0  ...   
...    ...                 ...    ...  ...  ...  ...  ...  ...  ...  ...  ...   
12249   11 2024-06-28 13:09:00      1    0    0    0    0    0    0    0  ...   
12250   11 2024-06-28 13:10:00      1    0    0    0    0    0    0    0  ...   
12251   11 2024-06-28 13:11:00      1    0    0    0    0    0    0    0  ...   
12252   11 2024-06-28 13:12:00      1    0    0    0    0    0    0    0  ...   
12253   11 2024-06-28 13:13:00      1    0    0    0    0    0    0    0  ...   

       Sp23  Sp24  Sp25  Sp26  Sp27  Sp28  Sp29  Sp30  Sp31  Sp32  
Index                                                              
19        0     0     0     0     0     0     0     0     0     0  
20        0     0     0     0     0     0     0     0     0     0  
21        0     0     0     0     0     0     0     0     0     0  
22        0     0     0     0     0     0     0     0     0     0  
23        0     0     0     0     0     0     0     0     0     0  
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  
12249     0     0     0     0     0     0     0     0     0     0  
12250     0     0     0     0     0     0     0     0     0     0  
12251     0     0     0     0     0     0     0     0     0     0  
12252     0     0     0     0     0     0     0     0     0     0  
12253     0     0     0     0     0     0     0     0     0     0  

[14254 rows x 35 columns]

In [56]:
# LS_path = "LS_" + path

# if not os.path.exists(LS_path):
#     os.makedirs(LS_path)

"""

for light in condition_keys:
    LS_period_approx = []
    LS_p_values = []
    for i in spiders:
        LS_app, p_value = period_LS(df, f"Sp{i}", light, 'value')
        LS_period_approx.append(LS_app)
        LS_p_values.append(p_value)
    print(light, LS_period_approx)
    print(light, LS_p_values)
    #data_report[f'LS {light}'] = LS_period_approx
    #data_report[f'LS {light} p-value'] = LS_p_values 
    


"""

x = "Sp25"

LS_period_approx = []

LS_period_approx.append(period_LS(df, x, 'LD', 'display'))
print(x, 'DD', " ", LS_period_approx, "\n\n\n")




"""

LS_period_approx = period_LS(df, x, 'DD', 'display')
print(x, 'DD', " ", LS_period_approx)

""" 



/opt/conda/envs/anaconda-2024.02-py310/lib/python3.10/site-packages/astropy/timeseries/periodograms/lombscargle/implementations/fast_impl.py:134: RuntimeWarning: invalid value encountered in divide
  power /= YY


ValueError: not enough values to unpack (expected 3, got 0)

# Suffering with Fourier

In [ ]:
from scipy.fft import fft, fftfreq

In [ ]:
curr_df = df[(df["Day"] >= 8) & (df["Day"] <= 15)]


activity = np.array(curr_df["Sp3"])
time = curr_df['Time'].apply(datetime_to_jd)

sampling_rate = 1440
n = len(activity)


fft_output = fft(activity)


freq_range = np.linspace(1/48, 2, len(activity))
frequencies = np.linspace(1/48, 2, len(activity))




# Compute power spectrum
power_spectrum = np.abs(fft_output)**2  # Truncate FFT output to match frequency range

# Find maximum power and corresponding frequency
max_power = np.max(power_spectrum)
max_freq = freq_range[np.argmax(power_spectrum)]



# Calculate period
period_hours = 1 / max_freq

# Plotting
fig, axs = plt.subplots(2, 1, figsize=(8, 6))

# Plot the original activity data
axs[0].plot(time, activity)
axs[0].set_title('Spider Activity Data')
axs[0].set_xlabel('Time (Julian Date)')
axs[0].set_ylabel('Activity')

# Plot the power spectrum
axs[1].plot(frequencies, power_spectrum)
axs[1].set_title(f'Power Spectrum (Period = {period_hours:.2f} hours)')
axs[1].set_xlabel('Frequency (cycles/hour)')
axs[1].set_ylabel('Power')

plt.tight_layout()
plt.show()











# Cosinor Period Estimate: Built in Version 

In [ ]:
import CosinorPy

In [ ]:
from CosinorPy import cosinor_nonlin
def cosinor_period_approx(df, sp):
    X = df['Time'].apply(datetime_to_jd).values
    Y = df[sp].values

    popt_ext, statistics, statistics_params = cosinor_nonlin.fit_generalized_cosinor(X,Y, period=None, min_per = 5, max_per=36, plot=True, plot_margins=False, save_to = "", x_label="time [h]", y_label="measurements", test="", hold_on=False, color="black")
    period_approx = statistics_params['params']['period']
    print(popt_ext, statistics, statistics_params)
    print('The approximated period is:', period_approx)

In [ ]:
cosinor_period_approx(df, 'Sp25')

# Cosinor Period Approximation: Periodogram Version

In [ ]:
#nonlinear cosinor model for period estimate 
from CosinorPy import cosinor_nonlin
from scipy import signal

def cosinor_period_approx_2(df, sp):
    X = df['Time'].apply(datetime_to_jd).values
    Y = df[sp].values
    _, _, statistics_params_basic = cosinor_nonlin.fit_cosinor_basic(X, Y, period=None)
    initial_period = statistics_params_basic['params']['period']
    initial_period_ci = statistics_params_basic['CIs']['period']
    
    # Step 2: Compute the periodogram and identify the dominant period
    frequency, power = LombScargle(X, Y).autopower()
    best_frequency = frequency[np.argmax(power)]
    best_period = 1 / best_frequency
    
    # Step 3: Refine the period estimate using fit_generalized_cosinor
    popt_ext, statistics, statistics_params = cosinor_nonlin.fit_generalized_cosinor(X, Y, period=best_period)
    
    # Print the results
    print("Initial Period Estimate (fit_cosinor_basic):", initial_period)
    print("Initial Period Confidence Interval:", initial_period_ci)
    print("Dominant Period from Periodogram:", best_period)
    print("Refined Period Estimate (fit_generalized_cosinor):", popt_ext[-1])

In [ ]:
cosinor_period_approx_2(df, 'Sp25')

# CSV dataframe

In [ ]:
data_report = pd.DataFrame(columns=['Group', 'SpiderN'])

group_info = [group_name for i in range(len(spiders))]

data_report['Group'] = group_info

data_report['SpiderN'] = spiders

group_spider = [g + str(s) for g, s in zip(group_info, spiders)]

data_report.insert(0, 'Spider', group_spider)

data_report = data_report.set_index('Spider')

data_report = data_report.drop(["Group", "SpiderN"], axis=1)

for light in condition_keys:
    LS_period_approx = []
    LS_p_values = []
    for i in spiders:
        LS_app, p_value = period_LS(df, f"Sp{i}", light, 'value')
        LS_period_approx.append(LS_app)
        LS_p_values.append(p_value)
    data_report[f'LS {light}'] = LS_period_approx
    data_report[f'LS {light} FAP'] = LS_p_values    




#for i, data in enumerate(new_data):
#    df.loc[len(df)] = [group_list[i]] + data[1:]

data_report

#period_LS(df, 'Sp17', 1, 7)

In [ ]:

"""

file_name = 'StA LD 1117-1128-2023.txt'

df, spiders = data_orgaziner(file_name)

display(df)

group_name, light_condition, start_date, path, two_lights = naming_pattern(file_name)

#print(group_name, light_condition, start_date, path, two_lights)


"""

folder_path = 'Data'
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]


for file_name in files:
    if file_name == "StA LD-DD 0118-0201-2024.txt":
        print(file_name, "\n")
        df, spiders = data_orgaziner(file_name)
        display(df)
        group_name, light_condition, start_date, path, two_lights = naming_pattern(file_name)
        condition_days, condition_keys = light_code(df)

        for light in condition_keys:
            for i in spiders:
                if i == 17:
                    fig = plt.figure(figsize=(4, 6))
                    raster_plot_2(f"Sp{i}", fig, light)

        LS_path = "LS_" + path

        if not os.path.exists(LS_path):
            os.makedirs(LS_path)
        
        for light in condition_keys:
            for i in spiders:
                LS_period_approx = period_LS(df, f"Sp{i}", light, 'value')
                print(i, light, " ", LS_period_approx)






